<a href="https://colab.research.google.com/github/shivag/cs145-public/blob/main/cs145_Fall22_V0_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### [Section 1] **Systems Primer**

We study a simplified IO model for HDDs and SSDs in cs145. The model will work well in practice, for our query optimzation and data layout problems. For more details on how the devices work, see **Optional reads** [How HDDs work?](https://pages.cs.wisc.edu/~remzi/OSTEP/file-disks.pdf) [How SSDs work?](https://pages.cs.wisc.edu/~remzi/OSTEP/file-ssd.pdf)

[Material from Lecture 1]



In [ ]:
#@title
import math

# We'll use MBs -- for basic conversion to MBs
MB = 1.0
GB = 1024.0
KB = 1.0/1024.0
Bytes = 1.0/(1024.0*1024.0)

# 64 MB-Blocks
PageSizeMB = 64.0

############
# Lecture 1: Access and Scan speeds in [seconds, MBps]
ssd = [10*pow(10, -6), 5.0*1024] # 10 microsecs, 5GBps
hdd = [10*pow(10, -3), 100.0]    # 10 millissecs, 100 MBps
IOdevices = {}

IOdevices['HDD'] = hdd
IOdevices['SSD'] = ssd

############
# Read costs: Simple cost model using Access time + Scan speeds (in cs145)
def readSpeed(numMBytes, blockSizeMBs, IOSpeed):
  (accessTime, scanSpeed) = IOSpeed

  # Assume you need to read full blocks. (Not optimized for partial blocks) 
  numBlocks = math.ceil(numMBytes/blockSizeMBs)
  tsecs = numBlocks*accessTime   # time to access
  tsecs += numMBytes/scanSpeed   # time to scan
  return tsecs

def writeSpeed(numMBytes, blockSizeMBs, IOSpeed):
  return readSpeed(numMBytes, blockSizeMBs, IOSpeed)

# Examples: 
# Read a 64MB block in a 64MB block
print('[Read SSD (secs)] 64MB block in 64MB-block SSD:', 
      readSpeed(64.0, PageSizeMB, ssd))
print('[Read HDD (secs)] 64MB block in 64MB-block HDD:', 
      readSpeed(64.0, PageSizeMB, hdd))

[Read SSD (secs)] 64MB block in 64MB-block SSD: 0.01251
[Read HDD (secs)] 64MB block in 64MB-block HDD: 0.65


### [Section 2] **ExternalSortMerge, SMJ, HPJ IO Costs**

[Material from Lectures 5, 6, 7, 8]

We need to model a real-world complex system. We use a simple model for good approximations of IO costs (and not get bogged down by other factors -- e.g., data caching, contentions in IO, etc.)  

In [ ]:
################################################
## IO Cost Equations
################################################
from math import ceil, log
# P(R) -- number of Pages for R
def POfTable(sizeInMBs):
  P = ceil(sizeInMBs/PageSizeMB)
  return P

# P(R) and B
def SortCost(P_R, B):
  # assume repacking optimization, and B-way merge. Assume B ~= B+1
  (R, W) = (1.0, 1.0) # R = 1 IO, W = 1 IO
  if P_R == 0:
    return 0
  return (R+W)*P_R*(ceil(log(P_R/(2*B), B)) + 1)  

#########
#### JOIN Algorithms
#### We'll handle OUT outside the function.
# P(R), P(S) and B: comute BNLJ IO Cost. 
def BNLJ(numPagesR, numPagesS, B):
  ioCost = numPagesR + numPagesR*ceil(numPagesS/B) 
  return ioCost

# P(R), P(S), B. Are R and S sorted? 
# Should we assume small backups (linear? Depends on duplicates, and small B)?  
def SMJCost(P_R, P_S, B, isSortedR, isSortedS, smallBackup):
  ioCost = 0.0
  if (not isSortedR): # then sort R
    ioCost += SortCost(P_R, B)
  if (not isSortedS): # then sort S
    ioCost += SortCost(P_S, B)
  if (smallBackup): # enuf B, and non-duplicates
    ioCost += P_R + P_S
  else:  # assume worst case for now
    ioCost += (P_R*P_S)
  return ioCost

# P(R), P(S), B. Are R and S hash partioned? 
# Should we assume few collisions (linear? Depends on hash function, and B)  
def HPJCost(P_R, P_S, B, isPartitionedR, isPartitionedS, fewCollisions):
  ioCost = 0.0
  if (not isPartitionedR): # then hash-partition R
    ioCost += 2*P_R
  if (not isPartitionedS): # then hash-partition S
    ioCost += 2*P_S
  if fewCollisions:
    ioCost += (P_R + P_S)
  else: # worst-case
    ioCost += P_R*P_S
  return ioCost

## Basic query optimizer
# DB checks cost of all plans. Picks lowest cost, and executes that plan.
def QueryOptimizer(P_R, P_S, 
                   isSortedR, isSortedS, 
                   isPartitionedR, isPartitionedS):
  ioCost = float("inf") # some big value, like infinity
  if isSortedR and isSortedS:
    ioCost = min(ioCost, SMJCost(P_R, P_S, B, 
                                 isSortedR, isSortedS))
  if isPartitionedR and isPartitionedS:
    ioCost = min(ioCost, HPJCost(P_R, P_S, B, 
                                 isPartitionedR, isPartitionedS))
  ioCost = min(ioCost, BNLJ(P_R, P_S, B))
  return ioCost

In [ ]:
################################
## Examples: change B, P(R), P(S) and see IOcost for Algorithms 
################################
from IPython.display import display, HTML
import pandas as pd

prettyGrid = []
def PrintIOCostOfAlgorithms(sizeTableS, sizeTableR):
  P_R = POfTable(sizeTableR)
  P_S = POfTable(sizeTableS)
  BufSizes = [100.0, 1000.0] # at 64 MBs/page, B = [6.4 GBs, 64GBs]
  for B in BufSizes:
    prettyPrint = []
    inputSet= "P(R)={}, P(S)={}, B ={}".format(P_R, P_S, B)
    prettyPrint.append(["", inputSet, "", ""])
    prettyPrint.append(["Sort(R)", "-", "-", SortCost(P_R, B)])
    prettyPrint.append(["BNLJ(R, S)", "-", "OUT +", BNLJ(P_R, P_S, B)])
    prettyPrint.append(["SMJ(R, S)", "Unsorted R and S, no backup", "OUT +", 
                        SMJCost(P_R, P_S, B, False, False, True)])
    prettyPrint.append(["SMJ(R, S)", "Sorted R and S, no backup", "OUT + ", 
                        SMJCost(P_R, P_S, B, True, True, True)])
    prettyPrint.append(["HPJ(R, S)", "HPed R and S, few collisions", "OUT +", 
                        HPJCost(P_R, P_S, B, False, False, True)])
    prettyPrint.append(["HPJ(R, S)", "HPed R and S, few collisions", "OUT +", 
                        HPJCost(P_R, P_S, B, True, True, True)])
    prettyGrid.append(pd.DataFrame(prettyPrint, 
                                   columns= ["Algorithm", "Conditions", "OutputCost", "IOCOst"]))

css = """
.output {
    flex-direction: row;
}
"""
HTML('<style>{}</style>'.format(css))
for scale in [1.0, 10.0]:
  sizeP = 100000.0 * MB * scale
  sizeR = 100000.0 * MB * scale
  print("*************** Size (MBs): R = {}, S = {}".format(sizeP, sizeR))
  PrintIOCostOfAlgorithms(sizeP, sizeR)
  for df in prettyGrid:
    display(df)
  prettyGrid = []


*************** Size (MBs): R = 100000.0, S = 100000.0


,Algorithm,Conditions,OutputCost,IOCOst
0,,"P(R)=1563, P(S)=1563, B =100.0",,
1,Sort(R),-,-,6252.0
2,"BNLJ(R, S)",-,OUT +,26571
3,"SMJ(R, S)","Unsorted R and S, no backup",OUT +,15630.0
4,"SMJ(R, S)","Sorted R and S, no backup",OUT +,3126.0
5,"HPJ(R, S)","HPed R and S, few collisions",OUT +,9378.0
6,"HPJ(R, S)","HPed R and S, few collisions",OUT +,3126.0


,Algorithm,Conditions,OutputCost,IOCOst
0,,"P(R)=1563, P(S)=1563, B =1000.0",,
1,Sort(R),-,-,3126.0
2,"BNLJ(R, S)",-,OUT +,4689
3,"SMJ(R, S)","Unsorted R and S, no backup",OUT +,9378.0
4,"SMJ(R, S)","Sorted R and S, no backup",OUT +,3126.0
5,"HPJ(R, S)","HPed R and S, few collisions",OUT +,9378.0
6,"HPJ(R, S)","HPed R and S, few collisions",OUT +,3126.0


*************** Size (MBs): R = 1000000.0, S = 1000000.0


,Algorithm,Conditions,OutputCost,IOCOst
0,,"P(R)=15625, P(S)=15625, B =100.0",,
1,Sort(R),-,-,62500.0
2,"BNLJ(R, S)",-,OUT +,2468750
3,"SMJ(R, S)","Unsorted R and S, no backup",OUT +,156250.0
4,"SMJ(R, S)","Sorted R and S, no backup",OUT +,31250.0
5,"HPJ(R, S)","HPed R and S, few collisions",OUT +,93750.0
6,"HPJ(R, S)","HPed R and S, few collisions",OUT +,31250.0


,Algorithm,Conditions,OutputCost,IOCOst
0,,"P(R)=15625, P(S)=15625, B =1000.0",,
1,Sort(R),-,-,62500.0
2,"BNLJ(R, S)",-,OUT +,265625
3,"SMJ(R, S)","Unsorted R and S, no backup",OUT +,156250.0
4,"SMJ(R, S)","Sorted R and S, no backup",OUT +,31250.0
5,"HPJ(R, S)","HPed R and S, few collisions",OUT +,93750.0
6,"HPJ(R, S)","HPed R and S, few collisions",OUT +,31250.0


**OBSERVATIONS**


*   Notice in Case 2, BNLJ(R, S) is cheaper than SMJ(R, S). Why? R and S are pretty small vs B.(100 GBs vs 64 GBs of RAM). Intuitively, makes sense. That is, if the data fits in RAM, effectively BNLJ will just process in RAM. And will not have the pre-processing (Sort or HP) overhead. 
*   In other cases, SMJ and HPJ do better than BNLJ. Especially, when P_R and P_S is big compared to B. SMJ and HPJ do even better, if they were pre-sorted or
pre-partitioned (perhaps for another query or index). 



In [ ]:
def BTree(numSearchKeys, SKeySize, PointerSize):
  f = int(PageSizeMB/(SKeySize+PointerSize))
  h = ceil(math.log(numSearchKeys, f))
  return (f, h)

dfList = []
for nSKeys in [pow(10, 7), pow(10, 9), pow(10, 12)]:
  for keySize in [4.0, 8.0, 128.0]: 
    # 4 and 8 are int/long. 128 for complex key, or k-dimensional key 
    (f, h) = BTree(nSKeys, keySize*Bytes, 8*Bytes)
    dfList.append([nSKeys, keySize, f, h])
display(pd.DataFrame(dfList, columns=["NumSearchKeys", 
                                      "Key Size (bytes)", "f", "h"]))


,NumSearchKeys,Key Size (bytes),f,h
0,10000000,4.0,5592405,2
1,10000000,8.0,4194304,2
2,10000000,128.0,493447,2
3,1000000000,4.0,5592405,2
4,1000000000,8.0,4194304,2
5,1000000000,128.0,493447,2
6,1000000000000,4.0,5592405,2
7,1000000000000,8.0,4194304,2
8,1000000000000,128.0,493447,3


### [Section 3] **Example Systems Design** -- CoOccuring Products [Lecture #9]

In [ ]:
(P_UserViews, P_ProductViews, P_CoOccur) = (3750.0, 5625.0, 563.0)

## Query to compute CoOccur
def ComputeCoOccur(B):
  # SMJ(UserViews, UserViews on userid). Self-Join => set SMJ's 2nd table to 0.0
  ioCost = SMJCost(P_UserViews, 0.0, B, False, False, True) + P_ProductViews # OUT
  
  # Handle groupbys: Sort(ProductViews)
  ioCost += SortCost(P_ProductViews, B) + P_CoOccur # OUT
  return ioCost

def IndexCoOccur(numSearchKeys):
  (f, h) = BTree(numSearchKeys, 4.0*Bytes, 8.0 * Bytes)
  print("B+ Tree -- f = {}, h = {}".format(f, h))

### Play with different B. Impact of B on IOCost 
Bs = [50.0, 100.0, 1000.0, 2000.0, 3000.0, 4000.0]
dfList = []
for B in Bs:
  ioCost = ComputeCoOccur(B)
  dfList.append([B, ioCost, 
                 readSpeed(ioCost*PageSizeMB, PageSizeMB, IOdevices['HDD']),
                           readSpeed(ioCost*PageSizeMB, 
                                     PageSizeMB, IOdevices['SSD'])])
display(pd.DataFrame(dfList, 
                     columns=["B", "IOCost", 
                              "HDD Time (secs)", "SSD Time (secs)"]))

IndexCoOccur(pow(10, 9))

,B,IOCost,HDD Time (secs),SSD Time (secs)
0,50.0,58688.0,38147.2,734.18688
1,100.0,47438.0,30834.7,593.44938
2,1000.0,47438.0,30834.7,593.44938
3,2000.0,39938.0,25959.7,499.62438
4,3000.0,28688.0,18647.2,358.88688
5,4000.0,28688.0,18647.2,358.88688


B+ Tree -- f = 5592405, h = 2


------------------------------------------------------------------------------
### [Section 4] **Transactions**

**Example: Update YouTube's numLikes for a video.**
 
How many updates-per-second can you run, for a large corpus of videos and user base?




In [ ]:
####
## Example for Youtube: update numLikes
#Need to update relevant row in SSD
timeSSD = writeSpeed(8.0*Bytes, PageSizeMB, ssd)
timeHDD = writeSpeed(8.0*Bytes, PageSizeMB, hdd)
numUpdatesPerSec = (1/timeSSD, 1/timeHDD)  # number of updates you can do per SSD, HDD
print(numUpdatesPerSec)

## That is, you can do up 100 updates/sec on HDD and 100,000 updates/sec on SSD.

(99985.10105892138, 99.9992370663676)


## [Section 4.1] **Big Idea: LOGs**

A **LOG file** (or LOG) is a special file managed by the DB. 
It's a separate file stored in your HDD or SSD. Works similar to a regular file. Except you only ``append'' to it, and its access cost is ~zero. (It's also called a **Ledger**. In some apps, a decentralized version is called a 'blockchain')

In [ ]:

# Negligible-cost (zero) access, for writes. E.g., when you append to a file.
def appendSpeed(numMBytes, blockSizeMBs, IOSpeed):
  (accessTime, scanSpeed) = IOSpeed
  # zero cost for access, using only scan speed (only append bytes needed)
  return numMBytes/scanSpeed

def speedupAppendVsWrite(numMBytes, blockSizeMBs, IOSpeed):
  return writeSpeed(numMBytes, blockSizeMBs, 
                    IOSpeed)/appendSpeed(numMBytes,blockSizeMBs, IOSpeed)


## 
## LOGs in HDDs:
## LOGs in SSDs: 


In [ ]:
###############################################
# Examples: 
# Append VS Writes for 16 bytes, 64 bytes to 1 MBs
# I.e., numLikes = numLikes+1, implies modifying 8 bytes in a row for a video 
import pandas as pd
from IPython.display import display, HTML

## Play around with these numbers for intuition
ex1BlockSize = [64.0*KB, 64.0*MB]  ## BlockSize or PageSize
ex1WriteNumMBs = [16.0*Bytes, 64.0*Bytes, 1.0*KB, 1.0*MB] # bytes to update 

def playWithBytesModified(writeBytes, blockSizes):
  print('--------Append vs Write:', writeBytes / Bytes, ' Bytes-----') 
  prettyPrint = []
  for iod in IOdevices.keys():
    for blks in blockSizes:
      tm = speedupAppendVsWrite(writeBytes, blks, IOdevices[iod])
      prettyPrint.append([iod, blks, tm])
  df = pd.DataFrame(prettyPrint,
                    columns = ["IODevice", "PageSize (MBs)", "IOCost"])
  return df

css = """
.output {
    flex-direction: row;
}
"""
HTML('<style>{}</style>'.format(css))
for numBytes in ex1WriteNumMBs:
  df = playWithBytesModified(numBytes, ex1BlockSize)
  display(df)

--------Append vs Write: 16.0  Bytes-----


,IODevice,PageSize (MBs),IOCost
0,HDD,0.0625,65537.0000
1,HDD,64.0000,65537.0000
2,SSD,0.0625,3356.4432
3,SSD,64.0000,3356.4432


--------Append vs Write: 64.0  Bytes-----


,IODevice,PageSize (MBs),IOCost
0,HDD,0.0625,16385.0000
1,HDD,64.0000,16385.0000
2,SSD,0.0625,839.8608
3,SSD,64.0000,839.8608


--------Append vs Write: 1024.0  Bytes-----


,IODevice,PageSize (MBs),IOCost
0,HDD,0.0625,1025.0000
1,HDD,64.0000,1025.0000
2,SSD,0.0625,53.4288
3,SSD,64.0000,53.4288


--------Append vs Write: 1048576.0  Bytes-----


,IODevice,PageSize (MBs),IOCost
0,HDD,0.0625,17.0000
1,HDD,64.0000,2.0000
2,SSD,0.0625,1.8192
3,SSD,64.0000,1.0512


**OBSERVATIONs**

In above example, observe for **Appends Vs Writes**  
1.    For small updates (e.g, 16 - 64 bytes), big speedup! (For both 64KB and 64MB PageSizes). Less dramatic for writing 1 MB. Makes intuitive sense -- for small updates, the Access cost dominates for Writes. 
2.   Specifically for 16 bytes, in **HDDs**, we get **62501x** speedup. In ** SSDs**, we get **3201x** speedup.


Why does this matter? Consider how to **edit (or update or modify or insert)** 16 bytes in a data row. (e.g., in YouTube, numLikes = numLikes+1) 
4. If we need to update 16 bytes for a row, it's much slower (**62501x** in HDDs, or **3201x** in SSDs) to update that value VERSUS **appending** the **updated** value (+ some extra information) into a **LOG**. (Conceptually, a TODO list of values to update in the right places.)
5. You can vary ex1MBsToWrite in above cell, from 4 bytes to 64 MBs, to see the speedup.



